In [21]:
%reload_ext autoreload
%autoreload 2

# OpenAI

In [2]:
#%%writefile "../openaigpt.py"
import os, sys

sys.path.append(os.path.expanduser("~/.django") )
import my_config
from my_config import *

from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key=my_config.OPENAI_KEY
)

'''response = client.completions.create(
    model="llama3.2",
    prompt="____",
    max_tokens=256
)

print(response.choices[0].text)
'''

completion = client.chat.completions.create(
  model = "llama3.2",
  messages=[
    {"role": "system"    , "content": "You are a helpful assistant."},
    {"role": "user"      , "content": "Who won the world series in 2020?"},
    {"role": "assistant" , "content": "The LA Dodgers won in 2020."},
    {"role": "user"      , "content": "Where was it played?"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='I can provide more specific information! The 2020 World Series was a "bubble" series, meaning all games were played within the MLB\'s COVID-19 neutral site bubble at Dodger Stadium in Los Angeles and Globe Life Field in Arlington, Texas (for the AL team). However, it was ultimately LA Dodgers vs Tampa Bay Rays', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


# OLLAMA

In [39]:
#%%writefile "../olgpt.py"
import sys, os, ollama, torch, logging,datetime
from ollama import Client
from mangorest.mango import webapi
sys.path.append(os.path.expanduser("~/.django") )
from openai import OpenAI
    
logger = logging.getLogger( "geoapp" )

device = "cpu"
if (torch.cuda.is_available() ):
    device = "cuda"


OLLAMA_HOST= 'http://10.0.0.29:11434/v1'

if (os.path.exists(os.path.expanduser("~/.django/my_config.py"))):
    import my_config
    try:
        from my_config import OLLAMA_HOST
    except:
        pass

client = OpenAI(
    base_url = OLLAMA_HOST,
    api_key=my_config.OPENAI_KEY
)

OLLAMA = Client(host=OLLAMA_HOST)
logger.info(f"OLLAMA_HOST => {OLLAMA_HOST}")
#--------------------------------------------------------------------------------------------------------    
@webapi("/gpt/generate/")
def ollma_generate(request=None, model="llama3|mistral", prompt="", stream=True,**kwargs):
    logger.info(f"Prepping: {datetime.datetime.now()}")
    model = model.split("|")[0].strip()
    response = OLLAMA.generate(model=model, prompt=prompt, stream=stream)
    # Stream response
    ret = ""
    for chunk in response:
        data = chunk["response"]
        ret += data
        
    logger.debug(ret)
    return ret

#--------------------------------------------------------------------------------------------------------    
@webapi("/gpt/openai/")
def openai(request=None, model="llama3.2|mistral", host="", use_openai="", prompt="", key="", messages="", **kwargs):

    if use_openai or model.startswith('gpt'):
         host= "https://api.openai.com/v1/"
    elif not host:
        host = OLLAMA_HOST
    
    client = OpenAI( base_url = host , api_key = key or my_config.OPENAI_KEY )

    tmsg = [
                {"role": "system"    , "content": "You are a helpful assistant."},
                {"role": "user"      , "content": prompt or "tell a joke?"}
    ]
    
    messages = messages or tmsg
    model    = model.split("|")[0]
    
    logger.info(f"Using host: {host}, {model} : {messages}")
    
    completion = client.chat.completions.create(
        model = model, #"llama3.2",
        messages= tmsg
        )

    ret = completion.choices[0].message
    
    return ret.content


Overwriting ../olgpt.py


In [15]:
#ollma_generate(prompt="whos is the chief data scientist of Space Lockheed Martin", stream=True)   
q1= "who is Shayma Alkobaisi from UAE"
q2="who won 1996 election"

q=q1
openai(prompt=q)

2024-11-02 11:59:23,107 geoapp INFO: Using host: , llama3.2 : [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'who is Shayma Alkobaisi from UAE'}]
2024-11-02 11:59:38,520 httpx INFO: HTTP Request: POST http://127.0.0.1:11434/v1/chat/completions "HTTP/1.1 200 OK"


"I couldn't find much information on a specific person named Shayma Alkobaisi from the UAE. It's possible that she's a private individual or not a public figure, which would explain the lack of available information.\n\nHowever, I did find some general information on people with similar names:\n\n* Yousef Al-Kobaisi is an Omani footballer who plays as a defender for Sur FC in Oman and has represented his national team.\n* Khalid Al-Kobaisi is also an Omani footballer who plays as a midfielder.\n\nPlease note that without more context or information about Shayma Alkobaisi, it's difficult to provide a specific answer. If you have any additional details, I'll do my best to help."

In [10]:
openai(prompt=q, model="gpt-3.5-turbo", host= "https://api.openai.com/v1/")

2024-11-02 11:54:53,562 geoapp INFO: Using host: https://api.openai.com/v1/, gpt-3.5-turbo : [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'who is Shayma Alkobaisi'}]
2024-11-02 11:54:53,828 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-11-02 11:54:53,829 openai._base_client INFO: Retrying request to /chat/completions in 0.419415 seconds
2024-11-02 11:54:55,184 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


"I'm sorry, but I couldn't find any relevant information on Shayma Alkobaisi. It is possible that she may not be a public figure or well-known individual. If you have any other questions or need assistance with something else, feel free to ask."

In [16]:
openai(prompt=q, model="gpt-4-0613", host= "https://api.openai.com/v1/")

2024-11-02 11:59:38,564 geoapp INFO: Using host: https://api.openai.com/v1/, gpt-4-0613 : [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'who is Shayma Alkobaisi from UAE'}]
2024-11-02 11:59:40,563 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


"I'm sorry, but there doesn't seem to be any public information available about a person named Shayma Alkobaisi from UAE. It's possible that you may be referring to a private individual or the spelling might be incorrect. If it's a non-public figure, it wouldn't be appropriate to share personal information about them. If it's a public figure, can you please provide more context or check the spelling of the name?"

# Experiments

In [11]:
import json
fg = '''  
messages=[
    {"role": "system"    , "content": "You are a helpful assistant."},
    {"role": "user"      , "content": "Who won the world series in 2020?"},
    {"role": "assistant" , "content": "The LA Dodgers won in 2020."},
    {"role": "user"      , "content": "Where was it played?"}
  ]
'''

#json.loads(fg)